In [19]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import pymorphy2
from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
from nltk.corpus import opinion_lexicon
nltk.download('opinion_lexicon')


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [164]:
df = pd.read_csv('Reviews.csv')
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [155]:
# df = df.head(10000)

In [166]:
df['negative'] = np.where(df['Score'] <= 2, 1, 0)
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,negative
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,0
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,1
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,0
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,1
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,0


In [167]:
df = df[['Text','negative']]
df

,Text,negative
0,I have bought several of the Vitality canned d...,0
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,0
3,If you are looking for the secret ingredient i...,1
4,Great taffy at a great price. There was a wid...,0
...,...,...
568449,Great for sesame chicken..this is a good if no...,0
568450,I'm disappointed with the flavor. The chocolat...,1
568451,"These stars are small, so you can give 10-15 o...",0
568452,These are the BEST treats for training and rew...,0


In [168]:
y = df['negative']
corpus = df['Text']

In [99]:
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    words = text.split() 
    lemms = list()
    for word in words:
        lemm = lemmatizer.lemmatize(word)
        lemms.append(lemm)

    return " ".join(lemms)

def clear_text(text):
    pattern = r'[^a-zA-Z]'
    c_text = re.sub(pattern, " ", text)
    
    return " ".join(c_text.split())

In [169]:
lemmatized_text = corpus.apply(lambda x: lemmatize(clear_text(x)))

In [111]:
def lemmatize2(text):
    words = text.split() 
    lemms = list()
    for word in words:
        lemm = lemmatizer.lemmatize(word)
        lemms.append(lemm)

    return lemms

def clear_text2(text):
    pattern = r'[^a-zA-Z]'
    c_text = re.sub(pattern, " ", text)
    
    return " ".join(c_text.split())

In [170]:
lemmatized_text2 = corpus.apply(lambda x: lemmatize2(clear_text(x)))

In [171]:
df['lemmatized_text']  = lemmatized_text
df['list_lemm']  = lemmatized_text2

In [172]:
df.head(10)

,Text,negative,lemmatized_text,list_lemm
0,I have bought several of the Vitality canned d...,0,I have bought several of the Vitality canned d...,"[I, have, bought, several, of, the, Vitality, ..."
1,Product arrived labeled as Jumbo Salted Peanut...,1,Product arrived labeled a Jumbo Salted Peanuts...,"[Product, arrived, labeled, a, Jumbo, Salted, ..."
2,This is a confection that has been around a fe...,0,This is a confection that ha been around a few...,"[This, is, a, confection, that, ha, been, arou..."
3,If you are looking for the secret ingredient i...,1,If you are looking for the secret ingredient i...,"[If, you, are, looking, for, the, secret, ingr..."
4,Great taffy at a great price. There was a wid...,0,Great taffy at a great price There wa a wide a...,"[Great, taffy, at, a, great, price, There, wa,..."
5,I got a wild hair for taffy and ordered this f...,0,I got a wild hair for taffy and ordered this f...,"[I, got, a, wild, hair, for, taffy, and, order..."
6,This saltwater taffy had great flavors and was...,0,This saltwater taffy had great flavor and wa v...,"[This, saltwater, taffy, had, great, flavor, a..."
7,This taffy is so good. It is very soft and ch...,0,This taffy is so good It is very soft and chew...,"[This, taffy, is, so, good, It, is, very, soft..."
8,Right now I'm mostly just sprouting this so my...,0,Right now I m mostly just sprouting this so my...,"[Right, now, I, m, mostly, just, sprouting, th..."
9,This is a very healthy dog food. Good for thei...,0,This is a very healthy dog food Good for their...,"[This, is, a, very, healthy, dog, food, Good, ..."


In [173]:
X_train, X_test, y_train, y_test = train_test_split(lemmatized_text, y)

In [174]:
# Преобразуйте коллекцию текстовых документов в матрицу счетчиков токенов.
count_vect = CountVectorizer(stop_words=stopwords)
n_gramm_train = count_vect.fit_transform(X_train)
n_gramm_test = count_vect.transform(X_test)


In [175]:
LR = LogisticRegression(max_iter=1000)
LR.fit(n_gramm_train, y_train)
print("\nF1:", f1_score(y_test, LR.predict(n_gramm_test)))


F1: 0.7220870866857341


In [182]:
print("\nF1:", accuracy_score(y_test, LR.predict(n_gramm_test)))


F1: 0.9268404238850501


In [176]:
# Эмоциональный словарь
pos_list=set(opinion_lexicon.positive())
neg_list=set(opinion_lexicon.negative())
print(len(pos_list))
print(len(neg_list))

2006
4783


In [177]:
def get_polarity_score(row):
    pos_count = 0
    neg_count = 0
    for word in row['list_lemm']:
        if word in pos_list:
            pos_count+=1
        if word in neg_list:
            neg_count+=1
    if neg_count>pos_count:
        return 1
    else:
        return 0

In [178]:
df['sen'] = df.apply(get_polarity_score, axis = 1)

In [179]:
df.head(20)

,Text,negative,lemmatized_text,list_lemm,sen
0,I have bought several of the Vitality canned d...,0,I have bought several of the Vitality canned d...,"[I, have, bought, several, of, the, Vitality, ...",0
1,Product arrived labeled as Jumbo Salted Peanut...,1,Product arrived labeled a Jumbo Salted Peanuts...,"[Product, arrived, labeled, a, Jumbo, Salted, ...",1
2,This is a confection that has been around a fe...,0,This is a confection that ha been around a few...,"[This, is, a, confection, that, ha, been, arou...",0
3,If you are looking for the secret ingredient i...,1,If you are looking for the secret ingredient i...,"[If, you, are, looking, for, the, secret, ingr...",0
4,Great taffy at a great price. There was a wid...,0,Great taffy at a great price There wa a wide a...,"[Great, taffy, at, a, great, price, There, wa,...",0
5,I got a wild hair for taffy and ordered this f...,0,I got a wild hair for taffy and ordered this f...,"[I, got, a, wild, hair, for, taffy, and, order...",0
6,This saltwater taffy had great flavors and was...,0,This saltwater taffy had great flavor and wa v...,"[This, saltwater, taffy, had, great, flavor, a...",0
7,This taffy is so good. It is very soft and ch...,0,This taffy is so good It is very soft and chew...,"[This, taffy, is, so, good, It, is, very, soft...",0
8,Right now I'm mostly just sprouting this so my...,0,Right now I m mostly just sprouting this so my...,"[Right, now, I, m, mostly, just, sprouting, th...",0
9,This is a very healthy dog food. Good for thei...,0,This is a very healthy dog food Good for their...,"[This, is, a, very, healthy, dog, food, Good, ...",0


In [180]:
print("\nF1:", f1_score(df['negative'], df['sen']))


F1: 0.39798734865536445


In [181]:
# Наиболее распространенным показателем для классификации является точность, 
# то есть доля выборок, правильно спрогнозированных, как показано ниже:
print("\nF1:", accuracy_score(df['negative'], df['sen']))


F1: 0.8461423439715439
